### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [ ]:
library(tibble)

In [ ]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

In [ ]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [ ]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [ ]:
head(df,2)

In [ ]:
df$flag <- FALSE
df$true_false <- FALSE


In [ ]:
head(df, 2)

## <font color = yellow>1. Create data & dfs

In [ ]:
create_data <- function () {   
   
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$date <- as.Date(df$date, origin = "1899-12-30")
 
    df$flag <- FALSE
    df$true_false <- FALSE
    
    sale_records <- data.frame() 

    # create dfs from filtered df
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    b <- buys[1,]
    s <- sells[1,]
    
    
    # Assign variables to the global environment
    assign("df", df, envir = .GlobalEnv)
    assign("buys", buys, envir = .GlobalEnv)
    assign("sells", sells, envir = .GlobalEnv)
    assign("b", b, envir = .GlobalEnv)
    assign("s", s, envir = .GlobalEnv)
    
    }

create_data()


In [ ]:
dim(df)


## <font color = lime> Examining data and dfs

In [ ]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 3)
head(sells,3)

In [ ]:
dim(b)
b

### <font color = yellow>2. update_b() - updates the basic b variable with data from buy/sell rows 
    
#### b>s, b<s, b==s,

In [ ]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

In [ ]:
last_row <- tail(buys, n = 1)
last_row

#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [ ]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function() {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) { 
            
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$sold_cf_pps <- b$pps
            b$sold_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE         
    
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {            
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
          
 #           s$type <- "SOLD"            
            }
            
        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

### <font color = yellow>3. create new sales_records (rbind) with the updated_b 

In [ ]:
df_name <- "iris"  # Replace with your data frame name
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)
         

In [ ]:
df_name <- "sale_records"
tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

In [ ]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    cat("Newly created sale_records with updated_b data. ", "\n")
    sale_records <- data.frame()
    sale_records <- rbind(sale_records, updated_b)    
}
sale_records

### <font color = yellow> 4. with_c/f, update buys or sells with a new row (add_row) when c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction


#### <font color = lime> How it's done

In [ ]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

In [ ]:
first_row <- head(sells, 1)
first_row

In [ ]:
dim(buys)
dim(sells)

In [ ]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_withcf <- function(buys, sells) {

    #  Check the most recent row ??? of sale_record or the last row of sale_record (was bind() or add_row() ? )
    last_row <- tail(sale_records, n = 1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    # both start with the same inputs from the sale_records
    b1 <- last_row
    s1 <- last_row
    
    
    
    
    
    
    
    #  b > s  ... buy c/f will be created 
   
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        
        
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
        
        
        
   
        
        
# -------------------------------------------------------------------------------------------------------------------------   
        
        
        # NEW  ... This section makes changes in sells[2, ]  which becomes sells[3,] , due to add_row()
                
        
        s2cf_quan <- sells$quan[2] - s1$sold_cf_quan
        s2cf_pps <- sells$pps[2]
        
        cat("the s2cf_quan is: ", s2cf_quan, "\n")
        cat("the s2cf_pps is ", s2cf_pps, "\n")
        
        sells$quan[2] <- s2cf_quan
        sells$pps[2] <-  s1$sell_pps                     #    s2cf_pps 
        sells[2, ]
        sells$basis[2] <- sells$pps[2] * s2cf_quan
        
       
        
# -------------------------------------------------------------------------------------------------------------------------    
        
        
        #  The section updates b1   .... t/f buys
        
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- FALSE
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        test <- test %>% add_row(b1, .before = 1)
        buys <- test
        
  # ---------------------------------------------------------------------------------------------------------------      
        
               
        # this section updates s1 ... t/f sells
       
        
            
        last_row$co <- "c/f"    
        
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <-  s1$sell_pps               #    s1$sold_cf_pps
        s1$basis <- s1$sell_pps
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
        
 
        
        
# =================================================================================================================
# ==============================================================================================================
        
        
             #   s > b ... sell carryforward will be created    
        

    
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
        
        
       
        cat("\n", "There is Sold carryforward. Update sells ", last_row$sold_cf_quan, "\n")
           # cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/f"      
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <- sells$pps[2]
        s1$basis <- s1$sold_cf_basis
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
        
        ## must also update the buys to match that sells c/f (this will allow for loop2 to begin 
        ## with both the buy and sell processing the c/f (example when sell cf = 900, then the 
        ## buy row needs to be that same 900 .... do not delete, just update here and now 
        
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- TRUE
        
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0
        
        test2 <- buys
        test2 <- test2 %>% add_row(b1, .before = 1)
        buys <- test2
                      
        }

    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ], dim of buys increases with c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... dim of sells increases with c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_withcf. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
return_list <- update_withcf(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)


#### <font color = yellow>5. Snip(), delete row from buys and sells / replace row when c/f 

In [ ]:
range(x <- sort(round(stats::rnorm(10) - 1.2, 1)))

if(any(x < 0)) cat("x contains negative values\n")

In [ ]:
if(any(buys[1,] == TRUE)) cat("buys contains cf \n")

In [ ]:
sale_records

### <font color = lime> When b == s, all is good above here

In [ ]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (buys, sells) {
   
    # parameters - rep the input vars (arguments)  ... may be extraneous
    b1 <- buys [1, ]
    s1 <- sells [1, ]
       
    # snip .... if 

    if(buys$type[1] == "Buy CF")   {         
        cat(" Contains buy carryforward - remove buys c/f row and DO snip sells \n")         
         # remove the original buys and sells
         buys <- buys[-2, ]    
         sells <- sells[-2, ]
        return(list(b1, s1, buys, sells))                   
        
    } else if (sells$type[1] == "Sell CF") {
        cat("sells contains carryforward .. Replace sells and DO snip buys \n") 
        sells <- sells[-2,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))    
    } else {
        cat("There is no carryforward in buys or sells, DO snip buys and Do snip sells \n")
        buys <- buys[-1,]
        sells <- sells[-1,]
        return(list(b1, s1, buys, sells))    
    }
}    
   
# fct call passing 2 arguments
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
# s1

#### <font color = lime> QC: Loop 1 review buys & sells

In [ ]:
buys[1:3,]
sells[1:3,]

#### <font color = lime> QC: Loop 1 review sale_record for accuracy

In [ ]:
sale_records

### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [ ]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} 

In [ ]:
#stopps #  Old code

# <font color = cyan> Proof of Concept above. Make it loop 2 x.

#### <font color = yellow>1. Create data & dfs

In [ ]:
b <- buys[1,]
s <- sells[1,]

### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [ ]:
updated_b <- update_b()
updated_b  
    

### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [ ]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
    sale_records <- rbind(sale_records, updated_b)
        
} else {
  # Data frame does not exist or is not a valid data frame
  # Handle the case where the data frame is missing...
    # cat("Newly created sale_records with updated_b data. ", "\n")
    # sale_records <- data.frame()
    # sale_records <- rbind(sale_records, updated_b)    
}


In [ ]:
sale_records

#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction

In [ ]:
# fct call
return_list <- update_withcf(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)


#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [ ]:
copy_snip_list <- copy_snip(buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [ ]:
sale_records

#### <font color = lime> QC: Loop 2 review buys & sells

In [ ]:
dim(buys)
dim(sells)
head(buys,3)
head(sells,3)

In [ ]:
stopps

#### <font color = lime> QC: review buys-sells for updated carryforward
    
#### When c/f exists, that c/f (type) must be inserted into the next row of either buys or sells